# Primary Text Processing

## Import

### Libraries

In [1]:
import os 
import codecs
from lxml import etree

from backend import *

### Definitions

In [2]:
texts = '../texts/fiction/'

libCols = ['author','pub_year','title','text']
tokenOHCO = ['w_id','part_num','para_num', 'sent_num', 'token_num']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', 'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

## Primary Texts

### Library

In [3]:
libDf = pd.DataFrame(columns = libCols)
for t in os.listdir(texts): 
    if t[-4:] == '.txt': 
        #print(t)
        info = re.match(r'(\w+)-(\d{4})-(.+).txt', t)
        with codecs.open(texts+t, 'r', encoding='windows-1251') as f: 
            textytext = f.read()
        libDf = libDf.append({
            'author': info.group(1),
            'pub_year': int(info.group(2)), 
            'title': info.group(3), 
            'text': textytext
        }, ignore_index=True)
        
libDf = libDf.sort_values(libCols[1:3]).reset_index().drop(['index'], axis=1)
libDf.index.name = 'w_id'
libTextsDf = libDf[[libCols[3]]]
libDf = libDf.drop(columns=[libCols[3]])
libDf

,author,pub_year,title
w_id,,,
0,gorkii,1900,troe
1,andreev,1903,zhizn-vasiliia-fiveiskogo
2,andreev,1904,gubernator
3,andreev,1905,k-zvezdam
4,andreev,1905,khristiane
5,andreev,1905,tak-bylo
6,gorkii,1906,mat
7,andreev,1906,savva-ignis-sanat
8,andreev,1907,iuda-iskariot


In [4]:
libTextsDf

,text
w_id,
0,Среди лесов Керженца рассеяно много оди...
1,*I*\n\n Над всей жизнью Василия ...
2,I \n\n \n ...
3,1\n \n Обсерват...
4,За окнами падал мокрый нояб...
5,1\n\n Стояла на площади огромная чер...
6,*I*\n\n Каждый день над рабочей слоб...
7,1\n \t \n В...
8,1\n \n Иисуса Христа много раз...


In [5]:
confessionDf = textRegularize(libTextsDf, 10)
#confessionDf

/home/xtra/code/diss/codebase/backend.py:172: FutureWarning: The default value of regex will change from True to False in a future version.
  textDf['text'] = textDf.text.str.replace('\n|\s{2,}', '')


In [6]:
dpDf = textRegularize(libTextsDf, 14)
#dpDf

In [7]:
motherTextDf = textRegularize(libTextsDf, 6)
#motherTextDf

In [8]:
# split into chapters
detstvoTextDf = libTextsDf.loc[[22]]
detstvoTextDf = pd.DataFrame(data=detstvoTextDf.text.str.split(r'\*\w+\*\n\n').to_list()[0]).reset_index()
detstvoTextDf = detstvoTextDf.rename(columns={'index':'chap', 0:'text'})[1:]
detstvoTextDf = detstvoTextDf.text.str.split('\n\n', expand=True).stack().to_frame()
#detstvoTextDf

### Tokens

In [9]:
motherTokenDf = pd.read_pickle('./proc/MotherTokendf.pkl')

In [10]:
motherTokenDf.set_index(['p_id','token_id'])

start stop       text head_id    rel   pos     lemma  anim  \
p_id token_id                                                              
1    1_1          0    6     Каждый     1_2    det   DET    каждый  None   
     1_2          7   11       день    1_27    obl  NOUN      день  Inan   
     1_3         12   15        над     1_5   case   ADP       над  None   
     1_4         16   23    рабочей     1_5   amod   ADJ   рабочий  None   
     1_5         24   33  слободкой    1_27    obl  NOUN  слободка  Inan   
...             ...  ...        ...     ...    ...   ...       ...   ...   
4436 1_1          0    6     Кто-то     1_2  nsubj  PRON    кто-то  None   
     1_2          7   14    ответил     1_0   root  VERB  ответить  None   
     1_3         15   17         ей     1_2   iobj  PRON       она  None   
     1_4         18   25    громким     1_5   amod   ADJ   громкий  None   
     1_5         26   34   рыданием     1_2    obl  NOUN   рыдание  Anim   

              aspect  case degree gender  mood number person tense verb_form  \
p_id token_id                                                                  
1    1_1        None   Acc   None   Masc  None   Sing   None  None      None   
     1_2        None   Acc   None   Masc  None   Sing   None  None      None   
     1_3        None  None   None   None  None   None   None  None      None   
     1_4        None   Ins    Pos    Fem  None   Sing   None  None      None   
     1_5        None   Ins   None    Fem  None   Sing   None  None      None   
...              ...   ...    ...    ...   ...    ...    ...   ...       ...   
4436 1_1        None   Nom   None   None  None   None   None  None      None   
     1_2        Perf  None   None   Masc   Ind   Sing   None  Past       Fin   
     1_3        None   Dat   None    Fem  None   Sing      3  None      None   
     1_4        None   Dat    Pos   None  None   Plur   None  None      None   
     1_5        None   Dat   None   Masc  None   Plur   None  None      None   

              voice  
p_id token_id        
1    1_1       None  
     1_2       None  
     1_3       None  
     1_4       None  
     1_5       None  
...             ...  
4436 1_1       None  
     1_2        Act  
     1_3       None  
     1_4       None  
     1_5       None  

[81373 rows x 18 columns]

In [11]:
pwd

'/home/xtra/code/diss/codebase'

In [13]:
from navec import Navec

path = './models/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)